# Please run:
pip install -r requirements.txt


In [2]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import random

df = pd.read_csv("cars.csv",encoding="cp1252")  # This made half of my hair turn gray

df.info()
df.describe()
df.isnull().sum()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Company Names              1218 non-null   object
 1   Cars Names                 1218 non-null   object
 2   Engines                    1218 non-null   object
 3   CC/Battery Capacity        1215 non-null   object
 4   HorsePower                 1218 non-null   object
 5   Total Speed                1218 non-null   object
 6   Performance(0 - 100 )KM/H  1212 non-null   object
 7   Cars Prices                1218 non-null   object
 8   Fuel Types                 1218 non-null   object
 9   Seats                      1218 non-null   object
 10  Torque                     1217 non-null   object
dtypes: object(11)
memory usage: 104.8+ KB


Company Names                0
Cars Names                   0
Engines                      0
CC/Battery Capacity          3
HorsePower                   0
Total Speed                  0
Performance(0 - 100 )KM/H    6
Cars Prices                  0
Fuel Types                   0
Seats                        0
Torque                       1
dtype: int64

In [3]:
print("df exists:", 'df' in locals())
print("Type of df:", type(df))
print(df.head())


df exists: True
Type of df: <class 'pandas.core.frame.DataFrame'>
  Company Names     Cars Names      Engines CC/Battery Capacity HorsePower  \
0       FERRARI  SF90 STRADALE           V8             3990 cc     963 hp   
1   ROLLS ROYCE        PHANTOM          V12             6749 cc     563 hp   
2          Ford            KA+  1.2L Petrol            1,200 cc   70-85 hp   
3      MERCEDES        GT 63 S           V8            3,982 cc     630 hp   
4          AUDI     AUDI R8 Gt          V10            5,204 cc     602 hp   

  Total Speed Performance(0 - 100 )KM/H      Cars Prices       Fuel Types  \
0    340 km/h                   2.5 sec      $1,100,000   plug in hyrbrid   
1    250 km/h                   5.3 sec        $460,000            Petrol   
2    165 km/h                  10.5 sec  $12,000-$15,000           Petrol   
3    250 km/h                   3.2 sec        $161,000            Petrol   
4    320 km/h                   3.6 sec        $253,290            Petrol   

  

In [4]:
df[df.isna().any(axis=1)]


,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
97,MERCEDES,BENZ EQS 53,ELECTRIC MOTOR,NaN,751 hp,250 km/h,3.4 sec,"$102,000",Electric,5,950 Nm
241,TOYOTA,COASTER,"4.0L,DIESEL","4,009 cc",134 hp,120 km/h,NaN,"$70,000",Diesel,20,500 Nm
255,Nissan,Urvan,2.5L Turbo Diese,NaN,2488 cc,140 km/h,14.0 sec,"$28,000",Diesel,15,356 Nm
1096,Mitsubishi,Canter (Truck),3.0L Diesel,"3,000 cc",150 hp,120 km/h,NaN,"$35,000 - $45,000",Diesel,2,370 Nm
1107,Mitsubishi,Fuso Fighter (Truck),7.5L Diesel,"7,500 cc",240 hp,120 km/h,NaN,"$70,000 - $90,000",Diesel,2,686 Nm
1108,Mitsubishi,Fuso Canter Eco Hybrid,3.0L Diesel Hybrid,"3,000 cc",150 hp,120 km/h,NaN,"$50,000 - $60,000",Diesel Hybrid,2,370 Nm
1126,Mitsubishi,Minicab MiEV,Electric Motor,16 kWh battery,64 hp,100 km/h,NaN,"$18,000 - $20,000",Electric,2,196 Nm
1147,Mazda,RX-Vision Concept,Rotary SkyActiv-R,NaN,400 hp (est.),300 km/h (est.),NaN,N/A (Concept Only),Petrol (Hybrid),2,NaN


# Cleaning

In [5]:

def power_clean(d):
    val = str(d["HorsePower"]).strip()
    
    # Replace any unusual range separators with a standard dash, no idea why it looks like there is 4 different dashes all out to ruin my day
    val = re.sub(r"[–—/]", "-", val)
    
    # Remove all characters except digits and dash
    val = re.sub(r"[^\d\-]", "", val)
    
    # If empty after cleaning, return NaN
    if not val:
        return np.nan
    
    # Handle ranges
    if "-" in val:
        parts = val.split("-")
        try:
            nums = [float(p) for p in parts if p]  # convert each part to float
            return sum(nums) / len(nums)  # average of range
        except:
            return np.nan
    else:
        try:
            return float(val)
        except:
            return np.nan

def price_clean(d):
    val = str(d["Cars Prices"]).strip()
    
    # Replace any unusual range separators with a standard dash, no idea why it looks like there is 4 different dashes all out to ruin my day
    val = re.sub(r"[–—/]", "-", val)
    
    # Remove all characters except digits and dash
    val = re.sub(r"[^\d\-]", "", val)
    
    # If empty after cleaning, return NaN
    if not val:
        return np.nan
    
    # Handle ranges
    if "-" in val:
        parts = val.split("-")
        try:
            nums = [float(p) for p in parts if p]  # convert each part to float
            return sum(nums) / len(nums)  # average of range
        except:
            return np.nan
    else:
        try:
            return float(val)
        except:
            return np.nan
perf = "Performance(0 - 100 )KM/H"
def perf_clean(d):
    val = str(d[perf]).strip()
    
    # Replace any unusual range separators with a standard dash, no idea why it looks like there is 4 different dashes all out to ruin my day
    val = re.sub(r"[–—/]", "-", val)
    
    # Remove all characters except digits and dash
    val = re.sub(r"[^\d\-]", "", val)
    
    # If empty after cleaning, return NaN
    if not val:
        return np.nan
    
    # Handle ranges
    if "-" in val:
        parts = val.split("-")
        try:
            nums = [float(p) for p in parts if p]  # convert each part to float
            return sum(nums) / len(nums)/10  # average of range, Div by 10 to fix numbers
        except:
            return np.nan
    else:
        try:
            return float(val)/10
        except:
            return np.nan


df[perf] = df.apply(perf_clean, axis=1)
def con_Speed(d):
    return d["Total Speed"].split(" ")[0]


perf = "Performance(0 - 100 )KM/H"
df[perf] = df.apply(perf_clean, axis=1)
df["Total Speed"] = df.apply(con_Speed,axis=1)
df["HorsePower"] = df.apply(power_clean, axis=1)
df["Cars Prices"] = df.apply(price_clean, axis=1)

df.to_csv("corrected.csv", index=False)
df


,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963.0,340,2.5,1100000.0,plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563.0,250,5.3,460000.0,Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",77.5,165,10.5,13500.0,Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630.0,250,3.2,161000.0,Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602.0,320,3.6,253290.0,Petrol,2,560 Nm
...,...,...,...,...,...,...,...,...,...,...,...
1213,Toyota,Crown Signia,2.5L Hybrid I4,2487 cc,240.0,180,7.6,45795.0,Hybrid (Gas + Electric),5,239 Nm
1214,Toyota,4Runner (6th Gen),2.4L Turbo I4 (i-FORCE MAX Hybrid),2393 cc + Battery,326.0,180,6.8,50000.0,Hybrid,7,630 Nm
1215,Toyota,Corolla Cross,2.0L Gas / 2.0L Hybrid,1987 cc / Hybrid batt,182.5,190,8.6,27172.5,Gas / Hybrid,5,190 – 210 Nm
1216,Toyota,C-HR+,1.8L / 2.0L Hybrid,1798 / 1987 cc + batt,169.0,180,9.2,33000.0,Hybrid,5,190 – 205 Nm


# Cleaning fuel types

In [6]:
df = pd.read_csv("corrected.csv",encoding="cp1252")

corrections = {
    # Specific corrections for misspellings

    "plug in hybrid": "Plug-in Hybrid",
    
    # Remove the fluff
    "petrol ": "Petrol",
    "hybrid (petrol)": "Hybrid",   # Standardizing all hybrids under one umbrella
    "petrol/hybrid": "Hybrid",
    "petrol, hybrid": "Hybrid",
    "hybrid/petrol": "Hybrid",
    "hybrid / plug-in": "Plug-in Hybrid",  # Plugins are not conventional hybrids
    "hybrid (gas + electric)": "Hybrid",
    "plug in hyrbrid": "Plug-in Hybrid",
    "diesel hybrid": "Hybrid",
    "hybrid/electric": "Hybrid",
    
    "petrol/diesel": "Petrol/Diesel",
    "diesel/petrol": "Petrol/Diesel",
    "petrol, diesel": "Petrol/Diesel",
    
    "petrol/ev": "Plug-in Hybrid",  
    "cng/petrol": "Petrol",         
    "petrol/awd": "Petrol",
    "plug-in hybrid": "Plug-in Hybrid"
}

df["Fuel Types"] = df["Fuel Types"].str.lower().str.strip().replace(corrections).str.title()

print(df["Fuel Types"].value_counts())

df.to_csv("corrected.csv", index=False)

df

Fuel Types
Petrol             874
Diesel             106
Hybrid             105
Electric            97
Petrol/Diesel       23
Plug-In Hybrid       8
Hydrogen             3
Petrol (Hybrid)      1
Gas / Hybrid         1
Name: count, dtype: int64


,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963.0,340,2.5,1100000.0,Plug-In Hybrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563.0,250,5.3,460000.0,Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",77.5,165,10.5,13500.0,Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630.0,250,3.2,161000.0,Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602.0,320,3.6,253290.0,Petrol,2,560 Nm
...,...,...,...,...,...,...,...,...,...,...,...
1213,Toyota,Crown Signia,2.5L Hybrid I4,2487 cc,240.0,180,7.6,45795.0,Hybrid,5,239 Nm
1214,Toyota,4Runner (6th Gen),2.4L Turbo I4 (i-FORCE MAX Hybrid),2393 cc + Battery,326.0,180,6.8,50000.0,Hybrid,7,630 Nm
1215,Toyota,Corolla Cross,2.0L Gas / 2.0L Hybrid,1987 cc / Hybrid batt,182.5,190,8.6,27172.5,Gas / Hybrid,5,190 â€“ 210 Nm
1216,Toyota,C-HR+,1.8L / 2.0L Hybrid,1798 / 1987 cc + batt,169.0,180,9.2,33000.0,Hybrid,5,190 â€“ 205 Nm
